In [1]:
# Supprimer l'ancien dossier corrompu
!rm -rf /content/drive/MyDrive/qwen_7b_model
print("✅ Ancien dossier supprimé")


✅ Ancien dossier supprimé


In [2]:
!pip install -q fastapi uvicorn pyngrok chromadb sentence-transformers
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q pydantic-settings python-dotenv httpx
print('✅ Dépendances installées')

✅ Dépendances installées


In [3]:
from google.colab import drive
drive.mount('/content/drive')
print('✅ Google Drive monté')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive monté


In [4]:
# Installer ngrok via snap (plus fiable sur Colab)
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
!sudo apt update -qq
!sudo apt install ngrok -qq

# Configurer le token
!ngrok authtoken 361IzWAOvuvUMfJjCWN0hOTPALb_7SPYyHQrzZkdHiXtNrnME

print('✅ ngrok installé et configuré')


deb https://ngrok-agent.s3.amazonaws.com buster main
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ngrok is already the newest version (3.33.1).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok installé et configuré


In [5]:
%%writefile database.py
import chromadb
from chromadb.config import Settings

DB_PATH = "/content/drive/MyDrive/conversation_db"

client = chromadb.PersistentClient(
    path=DB_PATH,
    settings=Settings(anonymized_telemetry=False)
)

conversations_collection = client.get_or_create_collection(
    name="conversations",
    metadata={"description": "Conversations metadata"}
)

messages_collection = client.get_or_create_collection(
    name="messages",
    metadata={"description": "Messages with embeddings"}
)

print(f"✅ ChromaDB initialisé: {DB_PATH}")

Overwriting database.py


In [6]:
%%writefile models.py
from dataclasses import dataclass, field
from typing import List, Optional
from datetime import datetime

@dataclass
class Conversation:
    id: str
    title: str
    mode: str
    created_at: str
    messages: List['Message'] = field(default_factory=list)

@dataclass
class Message:
    id: str
    conversation_id: str
    sender: str
    content: str
    timestamp: str
    suggestions: Optional[List[str]] = None

Overwriting models.py


In [7]:
%%writefile schemas.py
from pydantic import BaseModel
from typing import List, Optional

class MessageCreate(BaseModel):
    content: str

class MessageResponse(BaseModel):
    id: str
    sender: str
    content: str
    timestamp: str
    suggestions: Optional[List[str]] = None

class ConversationCreate(BaseModel):
    mode: str = "user_initiated"
    title: Optional[str] = "New Conversation"

class ConversationResponse(BaseModel):
    id: str
    title: str
    mode: str
    created_at: str
    messages: List[MessageResponse] = []

Overwriting schemas.py


In [8]:
%%writefile history_service.py
from typing import List
from datetime import datetime
import uuid
from models import Conversation, Message
from schemas import ConversationCreate
from database import conversations_collection, messages_collection

class HistoryService:
    async def create_conversation(self, conversation_data: ConversationCreate) -> Conversation:
        conv_id = str(uuid.uuid4())
        created_at = datetime.utcnow().isoformat()

        conversations_collection.add(
            ids=[conv_id],
            metadatas=[{
                "title": conversation_data.title,
                "mode": conversation_data.mode,
                "created_at": created_at
            }],
            documents=[f"{conversation_data.title}"]
        )

        return Conversation(
            id=conv_id,
            title=conversation_data.title,
            mode=conversation_data.mode,
            created_at=created_at,
            messages=[]
        )

    async def get_conversation(self, conversation_id: str) -> Conversation:
        conv_result = conversations_collection.get(ids=[conversation_id])
        if not conv_result['ids']:
            return None

        metadata = conv_result['metadatas'][0]
        messages = await self.get_messages(conversation_id)

        return Conversation(
            id=conversation_id,
            title=metadata['title'],
            mode=metadata['mode'],
            created_at=metadata['created_at'],
            messages=messages
        )

    async def add_message(self, conversation_id: str, sender: str, content: str,
                         embedding: List[float], suggestions=None) -> Message:
        msg_id = str(uuid.uuid4())
        timestamp = datetime.utcnow().isoformat()

        metadata = {
            "conversation_id": conversation_id,
            "sender": sender,
            "timestamp": timestamp
        }

        if suggestions and len(suggestions) > 0:
            metadata["suggestions"] = ",".join(suggestions)

        messages_collection.add(
            ids=[msg_id],
            embeddings=[embedding],
            metadatas=[metadata],
            documents=[content]
        )

        return Message(
            id=msg_id,
            conversation_id=conversation_id,
            sender=sender,
            content=content,
            timestamp=timestamp,
            suggestions=suggestions
        )

    async def get_messages(self, conversation_id: str) -> List[Message]:
        results = messages_collection.get(
            where={"conversation_id": conversation_id}
        )

        messages = []
        for i, msg_id in enumerate(results['ids']):
            metadata = results['metadatas'][i]
            sugg_str = metadata.get('suggestions')
            suggestions = sugg_str.split(',') if sugg_str else None

            messages.append(Message(
                id=msg_id,
                conversation_id=conversation_id,
                sender=metadata['sender'],
                content=results['documents'][i],
                timestamp=metadata['timestamp'],
                suggestions=suggestions
            ))

        messages.sort(key=lambda x: x.timestamp)
        return messages

    async def list_conversations(self, skip: int = 0, limit: int = 100) -> List[Conversation]:
        results = conversations_collection.get()

        conversations = []
        for i, conv_id in enumerate(results['ids']):
            metadata = results['metadatas'][i]
            conversations.append(Conversation(
                id=conv_id,
                title=metadata['title'],
                mode=metadata['mode'],
                created_at=metadata['created_at'],
                messages=[]
            ))

        conversations.sort(key=lambda x: x.created_at, reverse=True)
        return conversations[skip:skip+limit]

    async def delete_conversation(self, conversation_id: str):
        # Supprimer les messages
        messages_collection.delete(where={"conversation_id": conversation_id})
        # Supprimer la conversation
        conversations_collection.delete(ids=[conversation_id])

    async def rename_conversation(self, conversation_id: str, title: str) -> Conversation:
        conv = await self.get_conversation(conversation_id)
        if conv:
            conversations_collection.update(
                ids=[conversation_id],
                metadatas=[{
                    "title": title,
                    "mode": conv.mode,
                    "created_at": conv.created_at
                }],
                documents=[title]
            )
            conv.title = title
        return conv


Overwriting history_service.py


In [9]:
%%writefile chat_service.py
from typing import List, Tuple, Optional
from models import Message
from sentence_transformers import SentenceTransformer
import torch
import os


class ChatService:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.embedding_model = None

        def load_models(self):
          if self.model is None:
            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            import os

            # Cache Hugging Face sur Drive
            cache_dir = "/content/drive/MyDrive/huggingface_cache"
            os.environ['HF_HOME'] = cache_dir
            os.environ['TRANSFORMERS_CACHE'] = cache_dir

            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )

            # Vérifier si déjà en cache
            model_cache_path = f"{cache_dir}/models--Qwen--Qwen2.5-7B-Instruct"
            if os.path.exists(model_cache_path):
                print("📂 Chargement Qwen 7B depuis cache Drive...")
            else:
                print("📥 Téléchargement Qwen 7B (première fois, ~5-10 min)...")

            self.tokenizer = AutoTokenizer.from_pretrained(
                "Qwen/Qwen2.5-7B-Instruct",
                cache_dir=cache_dir
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                "Qwen/Qwen2.5-7B-Instruct",
                quantization_config=quantization_config,
                device_map="auto",
                cache_dir=cache_dir
            )
            print("✅ Qwen 7B chargé")

        if self.embedding_model is None:
            print("🔄 Chargement modèle embeddings...")
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("✅ Modèle embeddings chargé")


    def generate_embedding(self, text: str) -> List[float]:
        if self.embedding_model is None:
            self.load_models()
        return self.embedding_model.encode(text).tolist()

    async def generate_response(self, history: List[Message]) -> Tuple[str, Optional[List[str]]]:
        try:
            if self.model is None:
                self.load_models()

            messages = [
                {
                    "role": "system",
                    "content": "Tu es un assistant IA intelligent et amical. Réponds de manière naturelle et concise en français."
                }
            ]

            for msg in history:
                role = "user" if msg.sender == "user" else "assistant"
                messages.append({"role": role, "content": msg.content})

            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=256,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            response = self.tokenizer.decode(
                outputs[0][len(inputs.input_ids[0]):],
                skip_special_tokens=True
            )

            return response.strip(), None

        except Exception as e:
            print(f"❌ Erreur génération: {e}")
            return "Désolé, erreur technique.", None


chat_service = ChatService()


Overwriting chat_service.py


In [10]:
%%writefile chat_router.py
from fastapi import APIRouter, HTTPException
from typing import List
from schemas import ConversationCreate, ConversationResponse, MessageCreate, MessageResponse
from history_service import HistoryService
from chat_service import chat_service

router = APIRouter()
history_service = HistoryService()

@router.post("/conversations", response_model=ConversationResponse)
async def create_conversation(conversation: ConversationCreate):
    conv = await history_service.create_conversation(conversation)
    return ConversationResponse(
        id=conv.id,
        title=conv.title,
        mode=conv.mode,
        created_at=conv.created_at,
        messages=[]
    )

@router.get("/conversations", response_model=List[ConversationResponse])
async def list_conversations():
    convs = await history_service.list_conversations()
    return [
        ConversationResponse(
            id=c.id,
            title=c.title,
            mode=c.mode,
            created_at=c.created_at,
            messages=[]
        ) for c in convs
    ]

@router.get("/conversations/{conversation_id}", response_model=ConversationResponse)
async def get_conversation(conversation_id: str):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    return ConversationResponse(
        id=conv.id,
        title=conv.title,
        mode=conv.mode,
        created_at=conv.created_at,
        messages=[
            MessageResponse(
                id=m.id,
                sender=m.sender,
                content=m.content,
                timestamp=m.timestamp,
                suggestions=m.suggestions
            ) for m in conv.messages
        ]
    )

@router.post("/conversations/{conversation_id}/messages", response_model=MessageResponse)
async def send_message(conversation_id: str, message: MessageCreate):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    user_embedding = chat_service.generate_embedding(message.content)
    user_msg = await history_service.add_message(
        conversation_id, "user", message.content, user_embedding, None
    )

    history = await history_service.get_messages(conversation_id)
    ai_response, suggestions = await chat_service.generate_response(history)

    ai_embedding = chat_service.generate_embedding(ai_response)
    ai_msg = await history_service.add_message(
        conversation_id, "ai", ai_response, ai_embedding, None
    )

    return MessageResponse(
        id=ai_msg.id,
        sender=ai_msg.sender,
        content=ai_msg.content,
        timestamp=ai_msg.timestamp,
        suggestions=ai_msg.suggestions
    )


Overwriting chat_router.py


In [11]:
%%writefile chat_router.py
from fastapi import APIRouter, HTTPException
from typing import List
from schemas import ConversationCreate, ConversationResponse, MessageCreate, MessageResponse
from history_service import HistoryService
from chat_service import chat_service

router = APIRouter()
history_service = HistoryService()

@router.post("/conversations", response_model=ConversationResponse)
async def create_conversation(conversation: ConversationCreate):
    conv = await history_service.create_conversation(conversation)

    # Message d'accueil si mode AI
    if conversation.mode == "ai_initiated":
        greeting = "Bonjour ! Je suis votre assistant IA. Comment puis-je vous aider aujourd'hui ?"
        greeting_embedding = chat_service.generate_embedding(greeting)
        await history_service.add_message(conv.id, "ai", greeting, greeting_embedding, None)
        conv = await history_service.get_conversation(conv.id)

    return ConversationResponse(
        id=conv.id,
        title=conv.title,
        mode=conv.mode,
        created_at=conv.created_at,
        messages=[
            MessageResponse(
                id=m.id,
                sender=m.sender,
                content=m.content,
                timestamp=m.timestamp,
                suggestions=m.suggestions
            ) for m in conv.messages
        ]
    )

@router.get("/conversations", response_model=List[ConversationResponse])
async def list_conversations():
    convs = await history_service.list_conversations()
    return [
        ConversationResponse(
            id=c.id,
            title=c.title,
            mode=c.mode,
            created_at=c.created_at,
            messages=[]
        ) for c in convs
    ]

@router.get("/conversations/{conversation_id}", response_model=ConversationResponse)
async def get_conversation(conversation_id: str):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    return ConversationResponse(
        id=conv.id,
        title=conv.title,
        mode=conv.mode,
        created_at=conv.created_at,
        messages=[
            MessageResponse(
                id=m.id,
                sender=m.sender,
                content=m.content,
                timestamp=m.timestamp,
                suggestions=m.suggestions
            ) for m in conv.messages
        ]
    )

@router.post("/conversations/{conversation_id}/messages", response_model=MessageResponse)
async def send_message(conversation_id: str, message: MessageCreate):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    user_embedding = chat_service.generate_embedding(message.content)
    user_msg = await history_service.add_message(conversation_id, "user", message.content, user_embedding, None)

    history = await history_service.get_messages(conversation_id)
    ai_response, suggestions = await chat_service.generate_response(history)

    ai_embedding = chat_service.generate_embedding(ai_response)
    ai_msg = await history_service.add_message(conversation_id, "ai", ai_response, ai_embedding, None)

    return MessageResponse(
        id=ai_msg.id,
        sender=ai_msg.sender,
        content=ai_msg.content,
        timestamp=ai_msg.timestamp,
        suggestions=ai_msg.suggestions
    )

@router.delete("/conversations/{conversation_id}")
async def delete_conversation(conversation_id: str):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    await history_service.delete_conversation(conversation_id)
    return {"message": "Conversation supprimée"}

@router.patch("/conversations/{conversation_id}")
async def rename_conversation(conversation_id: str, title: str):
    conv = await history_service.get_conversation(conversation_id)
    if not conv:
        raise HTTPException(status_code=404, detail="Conversation not found")

    updated = await history_service.rename_conversation(conversation_id, title)
    return ConversationResponse(
        id=updated.id,
        title=updated.title,
        mode=updated.mode,
        created_at=updated.created_at,
        messages=[
            MessageResponse(
                id=m.id,
                sender=m.sender,
                content=m.content,
                timestamp=m.timestamp,
                suggestions=m.suggestions
            ) for m in updated.messages
        ]
    )


Overwriting chat_router.py


In [12]:
%%writefile main.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from chat_router import router

app = FastAPI(title="AI Conversation Backend - Colab")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(router)

@app.get("/")
async def root():
    return {"message": "Backend running on Colab with ChromaDB + Qwen 7B"}

Overwriting main.py


In [13]:
%%writefile chat_service.py
from typing import List, Tuple, Optional
from models import Message
from sentence_transformers import SentenceTransformer
import torch
import os

class ChatService:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.embedding_model = None

    def load_models(self):
        if self.model is None:
            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

            # Cache Hugging Face sur Drive
            cache_dir = "/content/drive/MyDrive/huggingface_cache"
            os.environ['HF_HOME'] = cache_dir
            os.environ['TRANSFORMERS_CACHE'] = cache_dir

            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )

            # Vérifier si déjà en cache
            model_cache_path = f"{cache_dir}/models--Qwen--Qwen2.5-7B-Instruct"
            if os.path.exists(model_cache_path):
                print("📂 Chargement Qwen 7B depuis cache Drive...")
            else:
                print("📥 Téléchargement Qwen 7B (première fois, ~5-10 min)...")

            self.tokenizer = AutoTokenizer.from_pretrained(
                "Qwen/Qwen2.5-7B-Instruct",
                cache_dir=cache_dir
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                "Qwen/Qwen2.5-7B-Instruct",
                quantization_config=quantization_config,
                device_map="auto",
                cache_dir=cache_dir
            )
            print("✅ Qwen 7B chargé")

        if self.embedding_model is None:
            print("🔄 Chargement modèle embeddings...")
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("✅ Modèle embeddings chargé")

    def generate_embedding(self, text: str) -> List[float]:
        if self.embedding_model is None:
            self.load_models()
        return self.embedding_model.encode(text).tolist()

    async def generate_response(self, history: List[Message]) -> Tuple[str, Optional[List[str]]]:
        try:
            if self.model is None:
                self.load_models()

            messages = [
                {
                    "role": "system",
                    "content": "Tu es un assistant IA intelligent et amical. Réponds de manière naturelle et concise en français."
                }
            ]

            for msg in history:
                role = "user" if msg.sender == "user" else "assistant"
                messages.append({"role": role, "content": msg.content})

            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=128,
                    do_sample=False,
                    pad_token_id=self.tokenizer.eos_token_id,
                    use_cache=True
                )

            response = self.tokenizer.decode(
                outputs[0][len(inputs.input_ids[0]):],
                skip_special_tokens=True
            )

            return response.strip(), None

        except Exception as e:
            print(f"❌ Erreur génération: {e}")
            return "Désolé, erreur technique.", None

chat_service = ChatService()


Overwriting chat_service.py


In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import asyncio

nest_asyncio.apply()

# Démarrer ngrok
public_url = ngrok.connect(8000)
print("\n" + "="*60)
print("🌐 URL PUBLIQUE DE TON BACKEND:")
print(f"   {public_url}")
print("="*60)
print("\n📝 Copie cette URL et mets-la dans ton frontend (App.tsx)")
print("\n🚀 Serveur en cours d'exécution...\n")

# Démarrer FastAPI avec asyncio
config = uvicorn.Config("main:app", host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
await server.serve()



🌐 URL PUBLIQUE DE TON BACKEND:
   NgrokTunnel: "https://overfew-subtriquetrous-jae.ngrok-free.dev" -> "http://localhost:8000"

📝 Copie cette URL et mets-la dans ton frontend (App.tsx)

🚀 Serveur en cours d'exécution...



✅ ChromaDB initialisé: /content/drive/MyDrive/conversation_db


INFO:     Started server process [44598]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     41.86.249.180:0 - "GET /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247 HTTP/1.1" 200 OK
📥 Téléchargement Qwen 7B (première fois, ~5-10 min)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Qwen 7B chargé
🔄 Chargement modèle embeddings...
✅ Modèle embeddings chargé


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     41.86.249.180:0 - "POST /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247/messages HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "POST /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247/messages HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "OPTIONS /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247 HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247 HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "OPTIONS /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "OPTIONS /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations/d3e5b7ba-f079-44c3-b676-cf8bee581247 HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "OPTIONS /conversations/bdcabd25-0598-427e-bf0a-cb190812222b HTTP/1.1" 200 OK
INFO:     41.86.249.180:0 - "GET /conversations/bdcabd25-0598-427e-bf0a-cb190812222b HTTP/1.